Sliding Window: https://doheon.github.io/%EC%BD%94%EB%93%9C%EA%B5%AC%ED%98%84/time-series/ci-3.lstm-post/

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from datetime import datetime, timedelta
from pykrx import stock

import random
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch import optim
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error 

import joblib

In [2]:
import matplotlib as mpl
import matplotlib.font_manager
[f.name for f in matplotlib.font_manager.fontManager.ttflist if 'Nanum' in f.name]
# 유니코드 깨짐 현상 해결
mpl.rcParams['axes.unicode_minus'] = False
# 나눔 고딕 폰트 적용
mpl.rcParams['font.family'] = 'NanumGothic'

# 1. 함수

## 슬라이딩 윈도우 설정

In [3]:
## window dataset을 생성하는 클래스
class windowDataset(Dataset):
    def __init__(self, y, input_window, output_window, stride):
        #총 데이터의 개수
        L = y.shape[0]
        #stride씩 움직일 때 생기는 총 sample의 개수
        num_samples = (L - input_window - output_window) // stride + 1

        #input과 output : shape = (window 크기, sample 개수)
        X = np.zeros([input_window*30, num_samples])
        Y = np.zeros([output_window*30, num_samples])

        for i in np.arange(num_samples):
            start_x = stride*i
            end_x = start_x + input_window
            X[:,i] = y[start_x:end_x].reshape(1, -1).flatten()

            start_y = stride*i + input_window
            end_y = start_y + output_window
            Y[:,i] = y[start_y:end_y].reshape(1, -1).flatten()

        X = X.reshape(X.shape[0], X.shape[1], 1).transpose((1,0,2))
        Y = Y.reshape(Y.shape[0], Y.shape[1], 1).transpose((1,0,2))
        self.x = X
        self.y = Y     
        self.len = len(X)
        
    def __getitem__(self, i):
        return self.x[i], self.y[i]
    def __len__(self):
        return self.len

input window, output window, stride를 입력받고 iw + ow만큼의 길이를 stride 간격으로 sliding하면서 데이터셋을 생성  
결과의 첫 번째 값으로는 input, 두 번째 값으로는 output이 출력

## 모델링

### 모델 함수 구현

* encoder: input을 통해 decoder에 전달할 hidden state 생성
* decoder: input의 마지막 값과 encoder에서 받은 hidden state를 이용하여 한 개의 값을 예측
* encoder decoder: 위의 두 모델을 합침. 원하는 길이의 아웃풋이 나올 때까지 decoder를 여러번 실행시켜서 최종 output을 생성. 원할한 학습을 위해 decoder의 input으로 실제 값을 넣는 teach forcing 구현

In [4]:
## input으로부터 입력을 받고 lstm을 이용하여 디코더에 전달할 hidden state 생성
class lstm_encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers = 1):
        super(lstm_encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first=True)

    def forward(self, x_input):
        lstm_out, self.hidden = self.lstm(x_input)
        return lstm_out, self.hidden

In [5]:
## sequence의 이전값 하나와, 이전 결과의 hidden state를 입력받아서 다음 값 하나를 예측
## 마지막에 fc layer를 연결해서 input size와 동일하게 크기를 맞춰줌
class lstm_decoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers = 1):
        super(lstm_decoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, input_size)           

    def forward(self, x_input, encoder_hidden_states):
        lstm_out, self.hidden = self.lstm(x_input.unsqueeze(-1), encoder_hidden_states)
        output = self.linear(lstm_out)
        
        return output, self.hidden

In [6]:
## 위의 두 모델 합치기
## 인코더를 한 번 실행시키고 인코더에서 전달받은 hidden state와 input의 마지막 값을 decoder에 전달해서 다음 예측값을 구함
## 여기서 나온 값과 hidden state를 반복적으로 사용해서 원하는 길이가 될 때까지 decoder 실행
class lstm_encoder_decoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(lstm_encoder_decoder, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        self.encoder = lstm_encoder(input_size = input_size, hidden_size = hidden_size)
        self.decoder = lstm_decoder(input_size = input_size, hidden_size = hidden_size)

    def forward(self, inputs, targets, target_len, teacher_forcing_ratio):
        batch_size = inputs.shape[0]
        input_size = inputs.shape[2]

        outputs = torch.zeros(batch_size, target_len*30, input_size)

        _, hidden = self.encoder(inputs)
        decoder_input = inputs[:,-1, :]
        
        #원하는 길이가 될 때까지 decoder를 실행한다.
        for t in range(target_len*30): 
            out, hidden = self.decoder(decoder_input, hidden)
            out =  out.squeeze(1)
            
            # teacher forcing을 구현한다.
            # teacher forcing에 해당하면 다음 인풋값으로는 예측한 값이 아니라 실제 값을 사용한다.
            if random.random() < teacher_forcing_ratio:
                decoder_input = targets[:, t, :]
            else:
                decoder_input = out
            outputs[:,t,:] = out
        return outputs

    # 편의성을 위해 예측해주는 함수도 생성한다.
    def predict(self, inputs, target_len):
        self.eval()
        inputs = inputs.unsqueeze(0)
        batch_size = inputs.shape[0]
        input_size = inputs.shape[2]
        outputs = torch.zeros(batch_size, target_len*30, input_size)
        _, hidden = self.encoder(inputs)
        decoder_input = inputs[:,-1, :]
        for t in range(target_len*30): 
            out, hidden = self.decoder(decoder_input, hidden)
            out =  out.squeeze(1)
            decoder_input = out
            outputs[:,t,:] = out
        return outputs.detach().numpy()[0,:,0]

### 모델 학습

In [7]:
device = torch.device('cpu')

In [8]:
def train(train_loader, ow):
    model = lstm_encoder_decoder(input_size=1, hidden_size=16).to(device)
    
    learning_rate=0.01
    epoch = 3000
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    criterion = nn.MSELoss()
    
    model.train()
    loss_avgs = []
    with tqdm(range(epoch)) as tr:
        for i in tr:
            total_loss = 0.0
            tf_ratio = max(0.6 - i*0.0001, 0)  # 0.6에서 teacher forcing을 점진적으로 줄여나감
#             tf_ratio = 0.6  # tf 조정 없이 0.6으로 통일할 때 사용
            for x,y in train_loader:
                optimizer.zero_grad()
                x = x.to(device).float()
                y = y.to(device).float()
                output = model(x, y, ow, tf_ratio).to(device)
                loss = criterion(output, y)
                loss.backward()
                optimizer.step()
                total_loss += loss.cpu().item()
            tr.set_postfix(loss="{0:.5f}".format(total_loss/len(train_loader)))
            loss_avgs.append(total_loss/len(train_loader))
    return model, loss_avgs

## 최종 예측 모델 함수

In [9]:
def forecasting_model(df, iw, ow, st):
    print('iw='+str(iw), 'ow='+str(ow), 'st='+str(st))

    ## 데이터 스케일링
    ## 데이터 분할
    # 마지막 일주일의 데이터를 예측하는 것이 목표
    scaler = MinMaxScaler()
    train_data = np.array(df)
    scaler.fit(train_data)
    train_np = scaler.transform(train_data)

    ## 슬라이딩 윈도우 설정
    df_train_dataset = windowDataset(train_np, input_window=iw, output_window=ow, stride=st)
    df_train_loader = DataLoader(df_train_dataset, batch_size=256)

    ## 모델 학습
    df_model, loss_avg = train(df_train_loader, ow)

#     ## 모델 저장
#     joblib.dump(df_model, './result/final/model/out'+str(ow)+'/stride'+str(st)+'/'+str(iw)+'days_model.pkl')

#     ## 예측
#     device = torch.device('cpu')
#     df_predict = df_model.predict(torch.tensor(train_np).reshape(-1,1).to(device).float(), target_len=ow)
    
#     ## 예측값 저장
#     standard = pd.to_datetime('2023-01-01')
#     df_predict_df = pd.DataFrame(scaler.inverse_transform(df_predict.reshape(ow, 30)), columns=company, index=pd.date_range(standard, standard+timedelta(days=ow-1)))
#     df_predict_df.to_csv('./result/final/predict/out'+str(ow)+'/stride'+str(st)+'/'+str(iw)+'days.csv', encoding='cp949')
    return df_model, loss_avg

# 2. 데이터 적용

In [15]:
stocks = pd.read_csv('./source/30/stock_top30.csv', encoding='cp949')
stocks['Date'] = pd.to_datetime(stocks['Date'])
stocks.set_index(['Date'], inplace=True)
stocks = stocks.loc[:'2022-12-31']

In [16]:
stocks

,삼성전자,SK하이닉스,LG화학,삼성전자우,삼성SDI,현대차,NAVER,기아,카카오,POSCO홀딩스,...,KT&G,하나금융지주,LG,고려아연,SK텔레콤,삼성전기,두산에너빌리티,엔씨소프트,HMM,S-Oil
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,31520.000000,26600.0,340000.000000,17220.000000,152500.000000,216000.000000,71413.000000,56300.000000,18801.0,360500.000000,...,78700.000000,36300.000000,63292.0,405000.000000,25967.0,101000.0,35077.000000,154000.0,144704.000000,105500.000000
2013-01-03,30860.000000,26650.0,342500.000000,17200.000000,155500.000000,206000.000000,69373.000000,54600.000000,18881.0,370000.000000,...,77000.000000,36700.000000,64048.0,406500.000000,25880.0,100000.0,37372.000000,155000.0,137183.000000,105500.000000
2013-01-04,30500.000000,26350.0,331000.000000,16920.000000,157000.000000,206000.000000,71099.000000,53600.000000,20131.0,367000.000000,...,76200.000000,36550.000000,63481.0,378000.000000,26484.0,99000.0,37564.000000,158500.0,135679.000000,105000.000000
2013-01-05,30466.666667,26200.0,329166.666667,17013.333333,155833.333333,206833.333333,72511.666667,53733.333333,20386.0,367666.666667,...,76733.333333,36633.333333,63607.0,379833.333333,26628.0,99500.0,37589.333333,158000.0,135779.333333,104166.666667
2013-01-06,30433.333333,26050.0,327333.333333,17106.666667,154666.666667,207666.666667,73924.333333,53866.666667,20641.0,368333.333333,...,77266.666667,36716.666667,63733.0,381666.666667,26772.0,100000.0,37614.666667,157500.0,135879.666667,103333.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,58100.000000,77000.0,628000.000000,52500.000000,624000.000000,158000.000000,181000.000000,64100.000000,54400.0,292000.000000,...,95100.000000,45300.000000,81900.0,565000.000000,49300.0,133000.0,14850.000000,432000.0,21850.000000,89100.000000
2022-12-28,56600.000000,76000.0,602000.000000,51300.000000,603000.000000,154000.000000,180000.000000,60900.000000,53600.0,291000.000000,...,91400.000000,42150.000000,79500.0,575000.000000,48200.0,134000.0,15700.000000,433500.0,20100.000000,86300.000000
2022-12-29,55300.000000,75000.0,600000.000000,50500.000000,591000.000000,151000.000000,177500.000000,59300.000000,53100.0,276500.000000,...,91500.000000,42050.000000,78100.0,564000.000000,47400.0,130500.0,15400.000000,448000.0,19550.000000,83400.000000


In [13]:
## df, iw, ow, st, name
iw = 15
ow = 7
# for ow in [4, 7, 10, 15, 30]:  # output
for st in [1, 2, 3]:  # stride
    tmp_model, loss_list = forecasting_model(stocks, iw, ow, st)
    joblib.dump(tmp_model, './result/final2/iw'+str(iw)+'_ow'+str(ow)+'_st'+str(st)+'_model.pkl')
    pd.DataFrame(loss_list, columns=['loss']).to_csv('./result/final2/iw'+str(iw)+'_ow'+str(ow)+'_st'+str(st)+'_loss.csv')

iw=15 ow=7 st=1


100%|██████████████████████████████████████████████████████████████| 3000/3000 [5:01:59<00:00,  6.04s/it, loss=0.78990]


iw=15 ow=7 st=2


100%|██████████████████████████████████████████████████████████████| 3000/3000 [2:47:29<00:00,  3.35s/it, loss=0.04144]


iw=15 ow=7 st=3


100%|██████████████████████████████████████████████████████████████| 3000/3000 [1:45:47<00:00,  2.12s/it, loss=0.04481]


In [16]:
## df, iw, ow, st, name
iw = 15
ow = 10
# for ow in [4, 7, 10, 15, 30]:  # output
for st in [1, 2, 3]:  # stride
    tmp_model, loss_list = forecasting_model(stocks, iw, ow, st)
    joblib.dump(tmp_model, './result/final2/iw'+str(iw)+'_ow'+str(ow)+'_st'+str(st)+'_model.pkl')
    pd.DataFrame(loss_list, columns=['loss']).to_csv('./result/final2/iw'+str(iw)+'_ow'+str(ow)+'_st'+str(st)+'_loss.csv')

iw=15 ow=10 st=1


100%|██████████████████████████████████████████████████████████████| 3000/3000 [6:43:35<00:00,  8.07s/it, loss=0.39409]


iw=15 ow=10 st=2


100%|██████████████████████████████████████████████████████████████| 3000/3000 [3:28:50<00:00,  4.18s/it, loss=0.03918]


iw=15 ow=10 st=3


100%|██████████████████████████████████████████████████████████████| 3000/3000 [2:24:48<00:00,  2.90s/it, loss=0.04366]


In [20]:
## df, iw, ow, st, name
iw = 30
ow = 30
# for ow in [4, 7, 10, 15, 30]:  # output
for st in [1, 2, 3]:  # stride
    tmp_model, loss_list = forecasting_model(stocks, iw, ow, st)
    joblib.dump(tmp_model, './result/final2/top30/iw'+str(iw)+'_ow'+str(ow)+'_st'+str(st)+'_model.pkl')
    pd.DataFrame(loss_list, columns=['loss']).to_csv('./result/final2/top30/iw'+str(iw)+'_ow'+str(ow)+'_st'+str(st)+'_loss.csv')

iw=30 ow=30 st=1


  0%|                                                                | 2/3000 [01:56<48:34:49, 58.34s/it, loss=0.06636]


KeyboardInterrupt: 